# DataFrames.jl 介紹 (二): Joins 與 Split-Apply-Combine Strategy

![](https://juliadata.github.io/DataFrames.jl/stable/assets/logo.png)

DataFrames.jl 官方網站: [https://juliadata.github.io/DataFrames.jl/stable/](https://juliadata.github.io/DataFrames.jl/stable/)

DataFrames.jl GitHub: [https://github.com/JuliaData/DataFrames.jl/blob/master/docs/src/index.md](https://github.com/JuliaData/DataFrames.jl/blob/master/docs/src/index.md)

## 0. 安裝

如果尚未安裝過 DataFrames.jl 的話, 執行 `Pkg.add()` 進行安裝

In [2]:
using Pkg
Pkg.add(PackageSpec(name="DataFrames", version="0.20.2"))

   Updating registry at `C:\Users\qwerz\.julia\registries\General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


Fetching: [========================================>]  100.0 % % %12.0 %14.9 %>                                ]  18.3 % [=========>                               ]  22.0 % [===========>                             ]  25.6 % %==============>                          ]  33.2 %.6 %Fetching: [=================>                       ]  41.0 %Fetching: [==================>                      ]  43.2 %44.3 % [====================>                    ]  48.7 %9 %]  62.4 %0 %]  69.9 %          ]  74.7 %78.5 %==================================>      ]  84.2 %]  97.7 % %

  Resolving package versions...
   Updating `C:\Users\qwerz\.julia\environments\v1.4\Project.toml`
 [no changes]
   Updating `C:\Users\qwerz\.julia\environments\v1.4\Manifest.toml`
 [no changes]


DataFrames 安裝的版本

In [3]:
Pkg.installed()["DataFrames"]

┌ Warning: Pkg.installed() is deprecated
└ @ Pkg D:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.4\Pkg\src\Pkg.jl:531


v"0.20.2"

## 1. 建立 DataFrame

In [4]:
using DataFrames

使用 column by column 的方式建立 DataFrame

In [17]:
# 使用建構子建立空的 DataFrame
df1 = DataFrame()

""
""


In [18]:
# 指定各個 column 及其值, 加入到 DataFrame 中
df1.ID = ["001", "002", "003", "005"]
df1.name = ["志明", "志玲", "大雄", "阿文"]

df1

,ID,name
,String,String
1,001,志明
2,002,志玲
3,003,大雄
4,005,阿文


In [19]:
df2 = DataFrame()

df2.ID = ["001", "002", "003", "006"]
df2.job = ["軟體工程師", "系統架構師", "專案經理", "測試工程師"]

df2

,ID,job
,String,String
1,001,軟體工程師
2,002,系統架構師
3,003,專案經理
4,006,測試工程師


## 2. Join

DataFrames.jl 實作了類似關連式資料庫的 Join 功能, 可進行兩個 DataFrame 之間的 Join, 包含了:
- Inner Join
- Left Join
- Right Join
- Outer Join
- Semi Join
- Anti Join
- Cross Join


下列是各種 Join 的程式範例.

預設的 Join, 結果包含符合條件的資料及所有欄位.

In [20]:
join(df1, df2, on=:ID)

,ID,name,job
,String,String,String
1,001,志明,軟體工程師
2,002,志玲,系統架構師
3,003,大雄,專案經理


### Inner Join

結果包含符合條件的資料及所有欄位.

In [21]:
join(df1, df2, on = :ID, kind = :inner)

,ID,name,job
,String,String,String
1,001,志明,軟體工程師
2,002,志玲,系統架構師
3,003,大雄,專案經理


### Left Join

結果包含 df1 所有資料, 但若未對應到的 df2 欄位值則填入 missing.

In [22]:
join(df1, df2, on = :ID, kind = :left)

,ID,name,job
,String,String,String⍰
1,001,志明,軟體工程師
2,002,志玲,系統架構師
3,003,大雄,專案經理
4,005,阿文,missing


### Right Join

結果包含 df2 所有資料, 但若未對應到的 df1 欄位值則填入 missing.

In [23]:
join(df1, df2, on = :ID, kind = :right)

,ID,name,job
,String,String⍰,String
1,001,志明,軟體工程師
2,002,志玲,系統架構師
3,003,大雄,專案經理
4,006,missing,測試工程師


### Outer Join

結果包含 df1 和 df2 所有資料, 未對應到的欄位值則填入 missing.

In [24]:
join(df1, df2, on = :ID, kind = :outer)

,ID,name,job
,String,String⍰,String⍰
1,001,志明,軟體工程師
2,002,志玲,系統架構師
3,003,大雄,專案經理
4,005,阿文,missing
5,006,missing,測試工程師


### Semi Join

跟 Inner Join 相同, 但是結果僅包含 df1 的欄位.

In [25]:
join(df1, df2, on = :ID, kind = :semi)

,ID,name
,String,String
1,001,志明
2,002,志玲
3,003,大雄


### Anti Join

結果包含僅存在於 df1 的資料, 且僅包含 df1 的欄位.

In [26]:
join(df1, df2, on = :ID, kind = :anti)

,ID,name
,String,String
1,005,阿文


### Cross Join

可視為 df1 $\times$ df2 的結果, 為 2 個 DataFrame 的乘積.

Cross Join 也是唯一一個不需要 key 值連接的 Join.

In [27]:
# makeunique 設為 true, 在 Join 時同名欄位會依序編號顯示在結果中
join(df1, df2, kind = :cross, makeunique = true)

,ID,name,ID_1,job
,String,String,String,String
1,001,志明,001,軟體工程師
2,001,志明,002,系統架構師
3,001,志明,003,專案經理
4,001,志明,006,測試工程師
5,002,志玲,001,軟體工程師
6,002,志玲,002,系統架構師
7,002,志玲,003,專案經理
8,002,志玲,006,測試工程師
9,003,大雄,001,軟體工程師


### Join 連接的 Key 值欄位名稱不一樣的話

In [28]:
df3 = DataFrame()

df3.IDnew = ["001", "002", "003"]
df3.city = ["台北市", "新竹市", "台中市"]

df3

,IDnew,city
,String,String
1,001,台北市
2,002,新竹市
3,003,台中市


在 on 參數指定要連結的 Key 欄位名稱.

In [29]:
join(df1, df3, on = :ID => :IDnew)

,ID,name,city
,String,String,String
1,001,志明,台北市
2,002,志玲,新竹市
3,003,大雄,台中市


### Indicator

使用 `indicator` 參數會在結果中加入一個 column 顯示資料是存在於 `left_only`, `right_only`, 或是 `both`

In [30]:
join(df1, df2, on = :ID, kind = :outer, indicator = :source)

,ID,name,job,source
,String,String⍰,String⍰,Categorical…
1,001,志明,軟體工程師,both
2,002,志玲,系統架構師,both
3,003,大雄,專案經理,both
4,005,阿文,missing,left_only
5,006,missing,測試工程師,right_only


### Validate

若要檢核要連接的 Key 值是否有重複, 可以使用 `validate` 參數, `validate` 的值可以為 tuple 或是 pair, 檢核 left 或是 right 資料來源.

下面的例子示範在 right 資料來源有重複的情況, 產生並提示錯誤訊息.

In [31]:
df4 = DataFrame()

df4.ID = ["001", "002", "003", "003", "006"]
df4.job = ["軟體工程師", "系統架構師", "專案經理", "測試工程師", missing]

df4

,ID,job
,String,String⍰
1,001,軟體工程師
2,002,系統架構師
3,003,專案經理
4,003,測試工程師
5,006,missing


In [32]:
join(df1, df4, on = :ID, kind = :outer, indicator = :source, validate = (true, true))

ArgumentError: ArgumentError: Merge key(s) in df2 are not unique. First duplicate at row 4

## 3. The Split-Apply-Combine Strategy

DataFrames.jl 以 `by()` 函式來實作 Split-Apply-Combine 策略 (大陸有人譯做 "化整為零策略"), `by()` 函式的動作包含了 groupby、map、combine, 依行 (column) 或列 (row) 進行資料處理; 或將資料分組之後, 進行獨立的操作; 然後將處理完成的片段進行組合.

In [33]:
using CSV, Statistics

In [34]:
df = CSV.read("../../data/iris.csv", categorical=true)
first(df, 5)

,SepalLength,SepalWidth,PetalLength,PetalWidth,Class
,Float64,Float64,Float64,Float64,String
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa


### 使用 "column => function" pair

下面範例是根據 Class 欄位, 產生各類別的花瓣長度平均值. 產生的欄位名稱會自動生成.

In [35]:
by(df, :Class, :PetalLength => mean)

,Class,PetalLength_mean
,String,Float64
1,Iris-setosa,1.464
2,Iris-versicolor,4.26
3,Iris-virginica,5.552


欄位名稱也可以用欄位 index 代替.

In [36]:
by(df, 5, 3 => mean)

,Class,PetalLength_mean
,String,Float64
1,Iris-setosa,1.464
2,Iris-versicolor,4.26
3,Iris-virginica,5.552


可以給予 Pair 名稱, 做為產生的欄位名稱.

下面的範例是將各 Class 的資料數命名為 `count`

In [37]:
by(df, :Class, count = :Class => length)

,Class,count
,String,Int64
1,Iris-setosa,50
2,Iris-versicolor,50
3,Iris-virginica,50


### 使用 "column => function" pair 的 Tuple 或 Vector

透過 `by()` 函式進行 groupby 時, "column => function" pair 可以有多組; 多組的 pair 也可以傳入 tuple 或 vector 類型, 如下面範例.

In [38]:
by(df, :Class, count = :Class => length, PetalLength_mean = :PetalLength => mean)

,Class,count,PetalLength_mean
,String,Int64,Float64
1,Iris-setosa,50,1.464
2,Iris-versicolor,50,4.26
3,Iris-virginica,50,5.552


使用 Tuple.

In [39]:
by(df, :Class, (:Class => length, :PetalLength => mean))

,Class,Class_length,PetalLength_mean
,String,Int64,Float64
1,Iris-setosa,50,1.464
2,Iris-versicolor,50,4.26
3,Iris-virginica,50,5.552


使用 vector.

In [40]:
by(df, :Class, [:Class => length, :PetalLength => mean])

,Class,Class_length,PetalLength_mean
,String,Int64,Float64
1,Iris-setosa,50,1.464
2,Iris-versicolor,50,4.26
3,Iris-virginica,50,5.552
